In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

Found 3984 images belonging to 26 classes.
Found 1886 images belonging to 26 classes.


## Import

In [8]:
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend



ModuleNotFoundError: No module named 'tensorflow.python'

## Hyperparameters

In [ ]:
# Hyperparameters
learning_rate = 0.002           # Laju pembelajaran untuk optimizer
label_smoothing_factor = 0.1
batch_size = 64                   # Jumlah sampel per batch saat training
num_epochs = 30                   # Jumlah iterasi penuh melalui dataset
num_classes = 26                 # Output kelas: A-Z (26 huruf)
# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 256  # Ukuran input image untuk MobileViT
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

## MobileViT utilities

The MobileViT architecture is comprised of the following blocks:

* Strided 3x3 convolutions that process the input image.
* [MobileNetV2](https://arxiv.org/abs/1801.04381)-style inverted residual blocks for
downsampling the resolution of the intermediate feature maps.
* MobileViT blocks that combine the benefits of Transformers and convolutions. It is
presented in the figure below (taken from the
[original paper](https://arxiv.org/abs/2110.02178)):


![](https://i.imgur.com/mANnhI7.png)

In [ ]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    """
    Mendefinisikan blok konvolusi dasar yang terdiri dari layer Conv2D dengan aktivasi Swish.

    Args:
        x: Tensor input ke blok konvolusi (biasanya output dari layer sebelumnya).
        filters: Jumlah filter konvolusi. Default 16.
        kernel_size: Ukuran kernel (filter) konvolusi (misal: 3 untuk 3x3). Default 3.
        strides: Langkah pergerakan kernel di atas input. Strides > 1 melakukan downsampling. Default 2.

    Returns:
        Tensor output setelah operasi konvolusi dan aktivasi.
    """
    # Membuat instance dari layer Konvolusi 2D
    conv_layer = layers.Conv2D(
        filters=filters,        # Jumlah filter
        kernel_size=kernel_size,# Ukuran kernel
        strides=strides,        # Langkah pergerakan kernel (downsampling jika > 1)
        activation=keras.activations.swish, # Fungsi aktivasi Swish (non-linear)
        padding="same",         # Padding untuk mempertahankan ukuran spasial (relatif terhadap stride)
    )
    # Menerapkan layer konvolusi ke tensor input 'x' dan mengembalikan hasilnya
    return conv_layer(x)

# Reference: https://github.com/keras-team/keras/blob/e3858739d178fe16a0c77ce7fab88b0be6dbbdc7/keras/applications/imagenet_utils.py#L413C17-L435

# Fungsi ini digunakan untuk menghitung padding yang benar, terutama saat melakukan downsampling
# dengan stride > 1, untuk memastikan dimensi output cocok dengan ekspektasi.
# Referensi implementasi berasal dari utilitas aplikasi Keras.
def correct_pad(inputs, kernel_size):
    """
    Menghitung tuple padding yang benar untuk operasi konvolusi/pooling.
    Digunakan untuk menangani dimensi spasial ganjil saat downsampling.

    Args:
        inputs: Tensor input yang akan diberi padding. Digunakan untuk mendapatkan ukuran spasialnya.
        kernel_size: Ukuran kernel (filter) atau jendela pooling. Dapat berupa integer atau tuple (tinggi, lebar).

    Returns:
        Tuple of tuples yang merepresentasikan padding yang perlu ditambahkan ke input.
        Format: ((padding_atas, padding_bawah), (padding_kiri, padding_kanan)).
    """
    # Menentukan indeks dimensi spasial (tinggi dan lebar) berdasarkan format data gambar.
    # Jika 'channels_first' (misal: (batch, channels, height, width)), dimensi spasial dimulai dari indeks 2.
    # Jika 'channels_last' (misal: (batch, height, width, channels)), dimensi spasial dimulai dari indeks 1.
    img_dim = 2 if backend.image_data_format() == "channels_first" else 1

    # Mendapatkan ukuran spasial input tensor (tinggi, lebar).
    # Slicing [img_dim : (img_dim + 2)] akan mengambil dimensi tinggi dan lebar.
    input_size = inputs.shape[img_dim : (img_dim + 2)]

    # Memastikan kernel_size adalah tuple (tinggi_kernel, lebar_kernel).
    # Jika diberikan sebagai integer tunggal, ubah menjadi tuple (integer, integer).
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)

    # Menghitung "penyesuaian" padding yang diperlukan jika dimensi input ganjil.
    # adjust[0] = 1 jika tinggi input ganjil, 0 jika genap.
    # adjust[1] = 1 jika lebar input ganjil, 0 jika genap.
    if input_size[0] is None:
        # Menangani kasus di mana dimensi input tidak diketahui pada waktu kompilasi (misal, None).
        # Asumsikan penyesuaian 1,1 (paling aman untuk downsampling pada dimensi ganjil).
        adjust = (1, 1)
    else:
        # Hitung sisa pembagian dengan 2. 1 - (dim % 2) akan menjadi 1 jika dim ganjil, dan 0 jika dim genap.
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)

    # Menghitung padding "benar" teoritis tanpa mempertimbangkan dimensi ganjil.
    # Untuk ukuran kernel K, padding yang umum adalah K // 2 (integer division).
    # Ini akan memberikan padding simetris jika input berukuran tepat untuk stride.
    correct = (kernel_size[0] // 2, kernel_size[1] // 2)

    # Mengembalikan tuple padding untuk diterapkan pada layer ZeroPadding2D.
    # Format output padding untuk ZeroPadding2D adalah ((padding_atas, padding_bawah), (padding_kiri, padding_kanan)).
    # Padding atas = correct[0] - adjust[0] (mengurangi 1 jika tinggi input ganjil)
    # Padding bawah = correct[0]
    # Padding kiri = correct[1] - adjust[1] (mengurangi 1 jika lebar input ganjil)
    # Padding kanan = correct[1]
    # Penyesuaian ini (mengurangi padding di satu sisi) memastikan output memiliki dimensi yang tepat setelah downsampling
    # ketika dimensi input awalnya ganjil.
    return (
        (correct[0] - adjust[0], correct[0]), # Padding (tinggi atas, tinggi bawah)
        (correct[1] - adjust[1], correct[1]), # Padding (lebar kiri, lebar kanan)
    )

# Reference: https://git.io/JKgtC

# Blok ini mengimplementasikan Inverted Residual Block, sebuah konsep dari MobileNetV2.
# Disebut "inverted" karena memperluas (expand) jumlah kanal di tengah blok,
# berbeda dengan blok residual tradisional yang menyempitkan (bottleneck) kanal.
def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    """
    Mendefinisikan Inverted Residual Block.

    Struktur: 1x1 Conv (Expand) -> 3x3 Depthwise Conv -> 1x1 Conv (Project)
    Ditambah Residual Connection jika dimensi input/output cocok dan strides=1.

    Args:
        x: Tensor input ke blok.
        expanded_channels: Jumlah kanal setelah layer ekspansi 1x1.
        output_channels: Jumlah kanal output setelah layer proyeksi 1x1.
        strides: Langkah untuk Depthwise Conv. Strides=2 melakukan downsampling. Default 1.

    Returns:
        Tensor output dari blok inverted residual.
    """
    # Simpan input tensor untuk residual connection nanti
    input_shape = x.shape

    # Langkah 1: Ekspansi (Expansion)
    # Konvolusi 1x1 untuk meningkatkan jumlah kanal
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m) # Menggunakan aktivasi Swish

    # Langkah 2: Depthwise Convolution
    # Konvolusi 3x3 yang beroperasi secara independen di setiap kanal
    # Jika strides=2, kita perlu padding khusus (menggunakan fungsi correct_pad)
    if strides == 2:
        # Menambahkan padding nol sebelum depthwise conv jika downsampling
        m = layers.ZeroPadding2D(padding=correct_pad(m, 3))(m)

    # Melakukan Depthwise Convolution
    m = layers.DepthwiseConv2D(
        kernel_size=3, # Ukuran kernel 3x3
        strides=strides, # Menggunakan stride yang ditentukan
        # Padding 'same' jika stride 1, 'valid' jika stride 2 (setelah ZeroPadding2D)
        padding="same" if strides == 1 else "valid",
        use_bias=False # Biasanya tanpa bias diikuti Batch Normalization
    )(m)
    m = layers.BatchNormalization()(m)
    m = keras.activations.swish(m) # Menggunakan aktivasi Swish

    # Langkah 3: Proyeksi (Projection)
    # Konvolusi 1x1 untuk mengurangi (memproyeksikan) jumlah kanal kembali
    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)
    # Catatan: Tidak ada aktivasi non-linear setelah proyeksi 1x1 di MobileNetV2,
    # ini disengaja untuk mempertahankan kemampuan merepresentasikan manifold berdimensi rendah.

    # Langkah 4: Residual Connection (jika memungkinkan)
    # Residual connection hanya ditambahkan jika:
    # 1. Jumlah kanal input sama dengan jumlah kanal output blok
    # 2. Tidak ada downsampling (strides == 1)
    # Ini memastikan dimensi spasial dan kanal cocok untuk operasi penjumlahan.
    # keras.ops.equal digunakan untuk perbandingan tensor.
    if keras.ops.equal(input_shape[-1], output_channels) and strides == 1:
        # Jika kondisi terpenuhi, tambahkan input asli ke output blok
        return layers.Add()([m, x])
    else:
        # Jika kondisi tidak terpenuhi (downsampling atau jumlah kanal berbeda),
        # kembalikan hanya output dari blok (tanpa residual connection)
        return m

# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/

# MLP block seperti yang sering digunakan dalam arsitektur Vision Transformer.
def mlp(x, hidden_units, dropout_rate):
    """
    Mendefinisikan blok Multi-Layer Perceptron (MLP) dengan aktivasi Swish dan Dropout.

    Args:
        x: Tensor input ke blok MLP.
        hidden_units: List integer yang menentukan jumlah unit (neuron)
                      di setiap layer Dense dalam MLP.
        dropout_rate: Tingkat dropout yang akan diterapkan setelah setiap layer Dense.

    Returns:
        Tensor output setelah melewati semua layer Dense dan Dropout.
    """
    # Iterasi melalui setiap jumlah unit yang ditentukan dalam list hidden_units
    for units in hidden_units:
        # Langkah 1: Layer Dense (Fully Connected)
        # Menambahkan layer Dense dengan jumlah unit tertentu dan aktivasi Swish.
        # Layer Dense menerapkan transformasi linier (x * W + b) diikuti oleh aktivasi.
        x = layers.Dense(units, activation=keras.activations.swish)(x)

        # Langkah 2: Layer Dropout
        # Menerapkan dropout untuk membantu mencegah overfitting.
        # Secara acak menyetel unit input ke 0 pada frekuensi 'dropout_rate' selama training. [1]
        x = layers.Dropout(dropout_rate)(x)

    # Mengembalikan tensor setelah melewati semua layer Dense dan Dropout dalam loop
    return x

# Blok Transformer seperti yang sering digunakan dalam arsitektur Vision Transformer.
def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    """
    Mendefinisikan satu atau lebih blok Transformer berturut-turut.

    Setiap blok terdiri dari:
    Layer Normalization -> Multi-Head Attention -> Skip Connection ->
    Layer Normalization -> MLP -> Skip Connection

    Args:
        x: Tensor input ke blok/rantai blok Transformer.
           Biasanya berbentuk (batch_size, num_patches, embedding_dim) atau serupa.
        transformer_layers: Jumlah blok Transformer untuk ditumpuk.
        projection_dim: Dimensi yang digunakan untuk key, query, dan value di Multi-Head Attention.
                        Juga merupakan dimensi input/output dari MLP.
        num_heads: Jumlah kepala perhatian di Multi-Head Attention. Default 2.

    Returns:
        Tensor output setelah melewati jumlah blok Transformer yang ditentukan.
    """
    # Loop untuk menumpuk (stack) beberapa blok Transformer
    for _ in range(transformer_layers):
        # Simpan input saat ini untuk skip connection pertama
        input_tensor = x

        # Layer Normalization 1.
        # Menormalisasi input di sepanjang dimensi terakhir (dimensi fitur/embedding).
        # Ini membantu menstabilkan training. epsilon ditambahkan untuk stabilitas numerik.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)

        # Create a multi-head attention layer.
        # Layer yang memungkinkan model untuk secara bersamaan memperhatikan dari representasi subspace yang berbeda.
        # Kueri (Query), Kunci (Key), dan Nilai (Value) semuanya berasal dari input yang sama (Self-Attention).
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,      # Jumlah kepala perhatian
            key_dim=projection_dim,   # Dimensi key/value per kepala perhatian
            dropout=0.1               # Tingkat dropout dalam mekanisme perhatian
        )(x1, x1) # Input Q, K, V adalah x1 (Self-Attention)

        # Skip connection 1.
        # Menambahkan output dari sub-layer attention ke input aslinya (sebelum normalisasi).
        # Ini membantu aliran gradien selama backpropagation.
        x2 = layers.Add()([attention_output, input_tensor]) # Menggunakan input_tensor yang disimpan

        # Layer Normalization 2.
        # Menormalisasi output dari skip connection perhatian.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

        # MLP (Feed-Forward Network).
        # Melewatkan output yang dinormalisasi melalui jaringan saraf terhubung penuh.
        # Menggunakan fungsi mlp yang didefinisikan sebelumnya.
        # Hidden units biasanya 2x dimensi input, lalu kembali ke dimensi input. [2]
        x3 = mlp(
            x3,
            hidden_units=[x.shape[-1] * 2, x.shape[-1]], # Pola umum untuk MLP di Transformer
            dropout_rate=0.1,
        )

        # Skip connection 2.
        # Menambahkan output dari sub-layer MLP ke inputnya (output dari skip connection pertama).
        x = layers.Add()([x3, x2]) # Menggunakan x2 sebagai input untuk skip connection ini

    # Mengembalikan tensor setelah melewati semua blok Transformer yang ditentukan
    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    """
    Mendefinisikan blok MobileViT yang menggabungkan konvolusi lokal dan pemrosesan global Transformer.

    Args:
        x: Tensor input ke blok MobileViT. Biasanya output dari blok sebelumnya.
        num_blocks: Jumlah blok Transformer yang akan digunakan di bagian global.
        projection_dim: Dimensi yang digunakan di dalam blok Transformer dan untuk konvolusi proyeksi.
        strides: Langkah untuk downsampling awal menggunakan conv_block. Default 1.

    Returns:
        Tensor output dari blok MobileViT yang telah memadukan fitur lokal dan global.
    """
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features

**More on the MobileViT block**:

* First, the feature representations (A) go through convolution blocks that capture local
relationships. The expected shape of a single entry here would be `(h, w, num_channels)`.
* Then they get unfolded into another vector with shape `(p, n, num_channels)`,
where `p` is the area of a small patch, and `n` is `(h * w) / p`. So, we end up with `n`
non-overlapping patches.
* This unfolded vector is then passed through a Tranformer block that captures global
relationships between the patches.
* The output vector (B) is again folded into a vector of shape `(h, w, num_channels)`
resembling a feature map coming out of convolutions.

Vectors A and B are then passed through two more convolutional layers to fuse the local
and global representations. Notice how the spatial resolution of the final vector remains
unchanged at this point. The authors also present an explanation of how the MobileViT
block resembles a convolution block of a CNN. For more details, please refer to the
original paper.

Next, we combine these blocks together and implement the MobileViT architecture (XXS
variant). The following figure (taken from the original paper) presents a schematic
representation of the architecture:

![](https://i.ibb.co/sRbVRBN/image.png)

In [ ]:
# Fungsi utama untuk membangun model MobileViT secara keseluruhan.
def create_mobilevit(num_classes=num_classes):
    """
    Membangun arsitektur model MobileViT.

    Model ini terdiri dari:
    - Layer input dan normalisasi awal.
    - Tahap-tahap yang menggabungkan Inverted Residual Blocks (MV2) dan MobileViT Blocks.
    - Downsampling dilakukan pada tahap-tahap awal dan di antara blok-blok utama.
    - Kepala klasifikasi di akhir.

    Args:
        num_classes: Jumlah kelas output untuk tugas klasifikasi. Default menggunakan variabel global num_classes.

    Returns:
        Instance model Keras Sequential atau Model.
    """
    # Mendefinisikan layer input model.
    # Model akan menerima input gambar dengan ukuran (image_size, image_size, 3)
    # image_size adalah variabel global yang didefinisikan sebelumnya (misal 256x256).
    # 3 menandakan channel warna RGB.
    inputs = keras.Input((image_size, image_size, 3))

    # Normalisasi awal: Menskalakan nilai piksel dari rentang [0, 255] ke [0.0, 1.0].
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Tahap Awal (Initial conv-stem -> MV2 block).
    # Memulai jaringan dengan blok konvolusi awal.
    x = conv_block(x, filters=16)
    # Diikuti oleh satu Inverted Residual Block (MobileNetV2 block) tanpa downsampling (stride default 1).
    # expanded_channels = 16 * expansion_factor, output_channels = 16.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=16
    )

    # Tahap Downsampling dengan MV2 block.
    # Melakukan downsampling pertama dengan Inverted Residual Block menggunakan strides=2.
    # expanded_channels = 16 * expansion_factor, output_channels = 24.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    # Diikuti oleh dua Inverted Residual Block tambahan tanpa downsampling.
    # expanded_channels = 24 * expansion_factor, output_channels = 24.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # Tahap Pertama MV2 -> MobileViT block.
    # Melakukan downsampling lagi dengan Inverted Residual Block menggunakan strides=2.
    # expanded_channels = 24 * expansion_factor, output_channels = 48.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    # Diikuti oleh blok MobileViT utama.
    # Menggunakan 2 blok Transformer di dalamnya (num_blocks=2).
    # Dimensi proyeksi fitur global adalah 64.
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Tahap Kedua MV2 -> MobileViT block.
    # Melakukan downsampling lagi dengan Inverted Residual Block menggunakan strides=2.
    # expanded_channels = 64 * expansion_factor, output_channels = 64.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    # Diikuti oleh blok MobileViT kedua.
    # Menggunakan 4 blok Transformer di dalamnya (num_blocks=4).
    # Dimensi proyeksi fitur global adalah 80.
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Tahap Ketiga MV2 -> MobileViT block.
    # Melakukan downsampling terakhir dengan Inverted Residual Block menggunakan strides=2.
    # expanded_channels = 80 * expansion_factor, output_channels = 80.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    # Diikuti oleh blok MobileViT ketiga.
    # Menggunakan 3 blok Transformer di dalamnya (num_blocks=3).
    # Dimensi proyeksi fitur global adalah 96.
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    # Menambahkan konvolusi 1x1 di akhir tahap fitur ekstraksi.
    # Filter=320, kernel=1, strides=1. Ini meningkatkan jumlah kanal sebelum kepala klasifikasi.
    x = conv_block(x, filters=320, kernel_size=1, strides=1)

    # Kepala Klasifikasi (Classification head).
    # Melakukan Average Pooling global 2D. Ini merata-ratakan fitur spasial di seluruh peta fitur,
    # mengurangi dimensi spasial menjadi 1x1.
    x = layers.GlobalAvgPool2D()(x)
    # Layer Dense (Fully Connected) terakhir untuk output klasifikasi.
    # Jumlah unit = num_classes.
    # Aktivasi "softmax" untuk menghasilkan probabilitas distribusi di atas kelas.
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    # Membuat model Keras menggunakan input dan output yang ditentukan.
    # Ini adalah Functional API Keras.
    return keras.Model(inputs, outputs)


# Membuat instance model MobileViT.
mobilevit_xxs = create_mobilevit()
# Menampilkan ringkasan model, termasuk layer dan jumlah parameter.
mobilevit_xxs.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        448 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │        512 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu (Silu)         │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 128, 128,  │        288 │ silu[0][0]        │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu_1 (Silu)       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │        512 │ silu_1[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 16)               │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │        512 │ add[0][0]         │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ silu_2 (Silu)       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 129, 129,  │          0 │ silu_2[0][0]      │
│ (ZeroPadding2D)     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_1  │ (None, 64, 64,    │        288 │ zero_padding2d[0

 Total params: 1,314,362 (5.01 MB)

 Trainable params: 1,311,818 (5.00 MB)

 Non-trainable params: 2,544 (9.94 KB)

## Load & Prepare Dataset

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Direktori data
train_dir = '/content/drive/MyDrive/PDM/Model/train'
val_dir = '/content/drive/MyDrive/PDM/Model/val'
#test_dir = '/content/drive/MyDrive/PDM/landmark-gambar/test'

# Augmentasi data training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=20,  # Rotasi acak hingga 20 derajat
    #width_shift_range=0.2,  # Pergeseran horizontal acak hingga 20% dari lebar
    #height_shift_range=0.2,  # Pergeseran vertikal acak hingga 20% dari tinggi
    #shear_range=0.2,  # Shear acak hingga 20 derajat
    #zoom_range=0.2,  # Zoom acak hingga 20%
    #horizontal_flip=True,  # Flip horizontal acak
    #fill_mode='nearest'  # Cara mengisi piksel kosong setelah transformasi
)

# Data generator untuk training
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Data generator untuk validasi (tanpa augmentasi)
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Data generator untuk testing (tanpa augmentasi)
#test_datagen = ImageDataGenerator(rescale=1./255)
#test_gen = test_datagen.flow_from_directory(
    #test_dir,
    #target_size=(img_size, img_size),
   # batch_size=batch_size,
  #  class_mode='categorical',
 #   shuffle=False
#)

In [ ]:

mobilevit_xxs.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


# Melakukan training dengan callback
history = mobilevit_xxs.fit(
    train_gen,
    epochs=num_epochs,
    validation_data=val_gen,
)

Epoch 1/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.0396 - loss: 3.2540 - val_accuracy: 0.0456 - val_loss: 3.2562
Epoch 2/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 514ms/step - accuracy: 0.0440 - loss: 3.2401 - val_accuracy: 0.0456 - val_loss: 3.2588
Epoch 3/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 526ms/step - accuracy: 0.0525 - loss: 3.2176 - val_accuracy: 0.0345 - val_loss: 3.2670
Epoch 4/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 523ms/step - accuracy: 0.0660 - loss: 3.1814 - val_accuracy: 0.0345 - val_loss: 3.3001
Epoch 5/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 539ms/step - accuracy: 0.0830 - loss: 3.0763 - val_accuracy: 0.0345 - val_loss: 3.4279
Epoch 6/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 32s 510ms/step - accuracy: 0.0769 - loss: 2.9839 - val_accuracy: 0.0345 - val_loss: 3.6596
Epoch 7/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 532ms/step - accuracy: 0.0916 - loss: 2.8998 - val_accuracy: 0.0345 - val_loss: 4.1730
Epoch 8/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 520ms/step - accuracy: 0.1231 - loss: 2.8436 - val_accura

## Train a MobileViT (XXS) model

In [ ]:


optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)


def run_experiment(epochs=num_epochs):
    mobilevit_xxs = create_mobilevit(num_classes=num_classes)
    mobilevit_xxs.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    # When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5`
    checkpoint_filepath = "/tmp/checkpoint.weights.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    mobilevit_xxs.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[checkpoint_callback],
    )
    mobilevit_xxs.load_weights(checkpoint_filepath)
    _, accuracy = mobilevit_xxs.evaluate(val_gen)
    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    return mobilevit_xxs


mobilevit_xxs = run_experiment()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.1692 - loss: 2.8714 - val_accuracy: 0.0345 - val_loss: 3.4748
Epoch 2/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 538ms/step - accuracy: 0.7014 - loss: 1.5729 - val_accuracy: 0.0371 - val_loss: 4.2392
Epoch 3/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 536ms/step - accuracy: 0.8459 - loss: 1.1597 - val_accuracy: 0.0382 - val_loss: 3.8285
Epoch 4/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 518ms/step - accuracy: 0.9021 - loss: 1.0059 - val_accuracy: 0.0371 - val_loss: 3.4876
Epoch 5/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 535ms/step - accuracy: 0.9342 - loss: 0.9047 - val_accuracy: 0.0345 - val_loss: 4.1114
Epoch 6/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 526ms/step - accuracy: 0.9493 - loss: 0.8554 - val_accuracy: 0.0345 - val_loss: 3.9230
Epoch 7/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 534ms/step - accuracy: 0.9647 - loss: 0.7999 - val_accuracy: 0.0350 - val_loss: 4.5343
Epoch 8/60
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 537ms/step - accuracy: 0.9728 - loss: 0.7860 - val_accura

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)


def run_experiment(epochs=num_epochs):
    mobilevit_xxs = create_mobilevit(num_classes=num_classes)
    mobilevit_xxs.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    # When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5`
    checkpoint_filepath = "/tmp/checkpoint.weights.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    mobilevit_xxs.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[checkpoint_callback],
    )
    mobilevit_xxs.load_weights(checkpoint_filepath)
    _, accuracy = mobilevit_xxs.evaluate(val_gen)
    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    return mobilevit_xxs


mobilevit_xxs = run_experiment()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 2785s 43s/step - accuracy: 0.1954 - loss: 2.8227 - val_accuracy: 0.0382 - val_loss: 3.5766
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 519ms/step - accuracy: 0.7538 - loss: 1.4534 - val_accuracy: 0.0371 - val_loss: 3.5051
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 538ms/step - accuracy: 0.8750 - loss: 1.1021 - val_accuracy: 0.0345 - val_loss: 3.4747
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 536ms/step - accuracy: 0.9150 - loss: 0.9632 - val_accuracy: 0.0345 - val_loss: 3.6915
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 537ms/step - accuracy: 0.9234 - loss: 0.9314 - val_accuracy: 0.0191 - val_loss: 5.8947
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 540ms/step - accuracy: 0.9560 - loss: 0.8301 - val_accuracy: 0.0382 - val_loss: 4.4632
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 535ms/step - accuracy: 0.9729 - loss: 0.7964 - val_accuracy: 0.0191 - val_loss: 4.0065
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 34s 539ms/step - accuracy: 0.9862 - loss: 0.7486 - val_accu

# Save Model

In [ ]:
mobilevit_xxs.save('/content/drive/MyDrive/PDM/model_ujicoba.keras')  # Change path as needed

In [ ]:
# Serialize the model as a SavedModel.
tf.saved_model.save(mobilevit_xxs, "mobilevit_xxs")

# Convert to TFLite. This form of quantization is called
# post-training dynamic-range quantization in TFLite.
converter = tf.lite.TFLiteConverter.from_saved_model("mobilevit_xxs")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS,  # Enable TensorFlow ops.
]
tflite_model = converter.convert()
open("mobilevit_xxs.tflite", "wb").write(tflite_model)

306464